In [27]:
import pandas as pd

# X_train = pd.read_csv('/workspaces/EDA_2/data/processed/clean_bank_X_train.csv')
# X_test = pd.read_csv('/workspaces/EDA_2/data/processed/clean_bank_X_test.csv')
y_train = pd.read_csv('/workspaces/EDA_2/data/processed/clean_bank_y_train.csv')
y_test = pd.read_csv('/workspaces/EDA_2/data/processed/clean_bank_y_test.csv')

X_train_norm = pd.read_csv('/workspaces/EDA_2/data/interim/clean_bank_X_train_std.csv')
X_test_norm = pd.read_csv('/workspaces/EDA_2/data/interim/clean_bank_X_test_std.csv')


Utilizamos f_regression ya que se trata de un problema de tipo numérico.

In [28]:
from sklearn.feature_selection import f_regression, SelectKBest

In [29]:
# With a value of k = 5 we implicitly mean that we want to remove 2 features from the dataset

selection_model = SelectKBest(f_regression, k = 5)
selection_model.fit(X_train_norm, y_train)
ix = selection_model.get_support()
X_train_sel = pd.DataFrame(selection_model.transform(X_train_norm), columns = X_train_norm.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test_norm), columns = X_test_norm.columns.values[ix])

X_train_sel.head()

/home/vscode/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,duration,pdays,emp.var.rate,euribor3m,nr.employed
0,-0.513276,0.196251,0.837830,0.772832,0.843441
1,0.148310,0.196251,-1.201789,-1.353534,-0.942293
2,0.086408,0.196251,0.837830,0.771100,0.843441
3,0.991736,-5.079513,-0.755622,-1.495561,-2.818006
4,0.469431,0.196251,0.837830,0.771677,0.843441


In [30]:
len(X_train_sel)

32950

In [31]:
X_train_sel["y"] = y_train
X_test_sel["y"] = y_test

In [32]:
X_train_sel.to_csv("/workspaces/EDA_2/data/interim/clean_bank_X_train_sel.csv", index=False)
X_test_sel.to_csv("/workspaces/EDA_2/data/interim/clean_bank_X_test_sel.csv", index=False)

En este paso trabajaremos el train y test con LogisticRegression con lo que nuestro objetivo final será obtener la precisión de nuestro modelo de Machine Learning. Será un modelo supervisado y decidimos pasarle todas las combinaciones de hiperparámetros para que la máquina determine los mejores. Al ser este un conjunto pequeño, no requerirá mucho tiempo realizar esta acción.

In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
train_data = X_train_sel
test_data = X_test_sel
train_data.head()

,duration,pdays,emp.var.rate,euribor3m,nr.employed,y
0,-0.513276,0.196251,0.837830,0.772832,0.843441,0
1,0.148310,0.196251,-1.201789,-1.353534,-0.942293,0
2,0.086408,0.196251,0.837830,0.771100,0.843441,0
3,0.991736,-5.079513,-0.755622,-1.495561,-2.818006,1
4,0.469431,0.196251,0.837830,0.771677,0.843441,0


In [35]:
model = LogisticRegression(random_state=24)
model

LogisticRegression(random_state=24)

In [36]:
X_train = train_data.drop(["y"], axis = 1)
y_train = train_data["y"]
X_train.head()

,duration,pdays,emp.var.rate,euribor3m,nr.employed
0,-0.513276,0.196251,0.837830,0.772832,0.843441
1,0.148310,0.196251,-1.201789,-1.353534,-0.942293
2,0.086408,0.196251,0.837830,0.771100,0.843441
3,0.991736,-5.079513,-0.755622,-1.495561,-2.818006
4,0.469431,0.196251,0.837830,0.771677,0.843441


In [37]:
y_train

0        0
1        0
2        0
3        1
4        0
        ..
32945    0
32946    0
32947    0
32948    0
32949    0
Name: y, Length: 32950, dtype: int64

In [38]:
X_test = test_data.drop(["y"], axis = 1)
y_test = test_data["y"]
X_test.head()

,duration,pdays,emp.var.rate,euribor3m,nr.employed
0,4.446688,0.196251,-1.138051,-1.666455,-2.430405
1,-0.451373,0.196251,-0.118241,0.304604,0.396315
2,-0.753150,0.196251,-0.118241,0.228394,0.396315
3,0.867931,0.196251,-1.902908,-1.591400,-1.259296
4,0.028374,0.196251,-1.201789,-1.716685,-2.193691


In [39]:
y_test

0       1
1       0
2       0
3       0
4       1
       ..
8233    1
8234    0
8235    0
8236    0
8237    0
Name: y, Length: 8238, dtype: int64

In [40]:
model.fit(X_train, y_train)

LogisticRegression(random_state=24)

In [41]:
y_pred = model.predict(X_train)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [42]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, y_pred)

0.9087101669195751

In [43]:
from sklearn.model_selection import GridSearchCV

hyperparams = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "penalty": ["l1", "l2", "elasticnet", None],
    "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
}

grid = GridSearchCV(model, hyperparams, cv = 5)
grid

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=24),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [44]:
grid.fit(X_train, y_train)

/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: U

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=24),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [45]:
grid.best_params_

{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}

In [46]:
grid.best_estimator_

LogisticRegression(C=1, penalty='l1', random_state=24, solver='liblinear')

In [47]:
best_model = grid.best_estimator_

In [48]:
y_pred = best_model.predict(X_train)

In [49]:
accuracy_score(y_train, y_pred)

0.908679817905918

In [50]:
y_pred_test = best_model.predict(X_test)

In [51]:
accuracy_score(y_test,y_pred_test)

0.9072590434571498

In [52]:
from pickle import dump

dump(best_model, open("/workspaces/EDA_2/models/bank_model.pk", "wb"))

## Conclusión:

La diferencia entre el accuracy del model con el best.model es apenas existente, siendo incluso la del best.model ligeramente inferior a la versión por defecto de LogicRegression. Finalmente nos quedamos con un 90% de precisión de nuestro modelo, lo cual valoramos como un porcentaje exitoso.
